In [1]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import torch
from torch import nn, optim
from torch.nn import functional as F
# import dataloader from torch
#import dataset
# import toTesor
import torchvision
from torchvision import transforms
from torchvision.io import read_image
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
from os.path import join
import sys
from PIL import Image
import cv2
from torchvision.transforms import Compose, RandomCrop, ToTensor, ToPILImage, CenterCrop, Resize

from dataset import TrainDatasetFromFolder
from model import Generator, Discriminator, GeneratorLoss
from tqdm import tqdm
from torch.autograd import Variable

c:\Users\Suren\.conda\envs\torch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# get psnr and ssim
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr


In [4]:
torch.autograd.set_detect_anomaly(True)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:

def get_factor(path):
    return int(path.split('_')[1])

factors = [get_factor(path) for path in os.listdir('models')]

In [5]:
generators = [Generator(factors[i]).to(device) for i in range(len(os.listdir('models')))]

In [6]:
for i, file in enumerate(os.listdir("models")):
    generators[i].load_state_dict(torch.load(join("models", file), map_location=device))
    generators[i].eval()

In [9]:
# run psnr and ssim
def run_psnr_ssim():
    # get test dataset
    test_dataset = TrainDatasetFromFolder('data/original/train', crop_size=256, upscale_factor=4)
    test_loader = DataLoader(dataset=test_dataset, num_workers=4, batch_size=1, shuffle=False)
    # get psnr and ssim
    psnr_list = []
    ssim_list = []
    for iteration, batch in enumerate(test_loader, 1):
        input, target = batch[0].to(device), batch[1].to(device)
        # do it for each generator
        for i in range(len(generators)):
            # get output
            output = generators[i](input)
            # get psnr and ssim
            psnr_list.append(psnr(target.cpu().detach().numpy(), output.cpu().detach().numpy()))
            ssim_list.append(ssim(target.cpu().detach().numpy(), output.cpu().detach().numpy(), multichannel=True))
    return psnr_list, ssim_list

In [ ]:
# graph psnr and ssim
def graph_psnr_ssim():
    psnr_list, ssim_list = run_psnr_ssim()
    # graph psnr
    plt.plot(psnr_list)
    plt.title('PSNR')
    plt.show()
    # graph ssim
    plt.plot(ssim_list)
    plt.title('SSIM')
    plt.show()

In [ ]:
graph_psnr_ssim()